In [32]:
import getpass
import os
password = getpass.getpass()
command = "sudo -S apt update"
os.system('echo %s | %s' % (password, command))

command = "sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null"
os.system('echo %s | %s' % (password, command))
!
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
if not os.path.exists("spark-3.5.3-bin-hadoop3.tgz"):
    !wget https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
%pip install -q findspark
%pip install pyspark
%pip install py4j
%pip install requests
%pip install dash
%pip install networkx
%pip install plotly


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...
Building dependency tree...
Reading state information...
All packages are up to date.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:

import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.5.3-bin-hadoop3"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Projekt grupowy") \
       .getOrCreate()

spark

In [34]:
search_phrase = '5g'

api_call = f'https://dblp.org/search/publ/api?q={search_phrase}&format=json'

In [35]:
try:
    # Your Spark code that might throw an exception
    spark.sql("""
        CREATE TABLE IF NOT EXISTS authors (pid string, author string) USING PARQUET
    """)
except Exception as e:
    print(f"Table already exists. Error code: {e}")


In [36]:
try:
    spark.sql("""CREATE TABLE IF NOT EXISTS publications (title string, authors_pid array<string>, year string) USING PARQUET""")
except Exception as e:
    print(f"Table already exists. Error code: {e}")

In [37]:
import requests
import pyspark.sql
authors = []
publications = []
data = requests.get(api_call).json()
data

{'result': {'query': '5g',
  'status': {'@code': '200', 'text': 'OK'},
  'time': {'@unit': 'msecs', 'text': '1.48'},
  'completions': {'@total': '1',
   '@computed': '1',
   '@sent': '1',
   'c': {'@sc': '19394',
    '@dc': '18814',
    '@oc': '19394',
    '@id': '57184699',
    'text': '5g'}},
  'hits': {'@total': '18814',
   '@computed': '100',
   '@sent': '30',
   '@first': '0',
   'hit': [{'@score': '4',
     '@id': '1021838',
     'info': {'authors': {'author': {'@pid': '214/4100',
        'text': 'Thomas Deinlein'}},
      'title': 'Simulation und Modellierung von 5G Vehicle-to-Infrastructure (V2I) / Vehicle-to-Network (V2N) Anwendungsfällen unter Berücksichtigung des 5G QoS Modells (Simulation and modeling of 5G Vehicle-to-Infrastructure (V2I) / Vehicle-to-Network (V2N) use cases under consideration of the 5G QoS model)',
      'year': '2022',
      'type': 'Books and Theses',
      'access': 'closed',
      'key': 'phd/basesearch/Deinlein22',
      'ee': 'https://opus4.kobv.de/

In [38]:
len(data['result']['hits']['hit'])

30

In [39]:
publications = []
authors = []

In [40]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType
authors_schema = StructType([
    StructField('pid', StringType(), True),
    StructField('author', StringType(), True),
])

publications_schema = StructType([
    StructField('title', StringType(), True),
    StructField('authors_pid', ArrayType(StringType()), True),
    StructField('year', StringType(), True)
])


for x in range(len(data['result']['hits']['hit'])):
  hit = data['result']['hits']['hit'][x]

  try:
    if hit['info'].get('authors') is None:
      continue
    if isinstance(hit['info']['authors']['author'], list):
      pid = []
      for y in range(len(hit['info']['authors']['author'])):
        authors.append([hit['info']['authors']['author'][y]['@pid'], hit['info']['authors']['author'][y]['text']])
        pid.append(hit['info']['authors']['author'][y]['@pid'])
      publications.append([hit['info']['title'], pid, hit['info']['year']])
    elif isinstance(hit['info']['authors']['author'], dict):
      authors.append([hit['info']['authors']['author']['@pid'], hit['info']['authors']['author']['text']])
      publications.append([hit['info']['title'], [hit['info']['authors']['author']['@pid']], hit['info']['year']])
  except Exception as e:
    print('Error: ', e)


authors_df = spark.createDataFrame(data=authors, schema=authors_schema)
publications_df = spark.createDataFrame(data=publications, schema=publications_schema)

In [41]:
try:
    authors_df.write.mode('append').saveAsTable("authors")
except Exception as e:
    print(f"Error code: {e}")


In [42]:
try:
    publications_df.write.mode('append').saveAsTable("publications")
except Exception as e:
    print(f"Error code: {e}")

In [43]:
from pyspark.sql.functions import array_contains
authors_df.join(publications_df, array_contains(publications_df.authors_pid, authors_df.pid), 'inner').select("pid", "author", "title", "year").show(1000)

+-----------------+--------------------+--------------------+----+
|              pid|              author|               title|year|
+-----------------+--------------------+--------------------+----+
|         214/4100|     Thomas Deinlein|Simulation und Mo...|2022|
|         282/9003|       Anouar Rkhami|From servies plac...|2021|
|         185/5677|Konstantinos Kousias|Proceedings of th...|2024|
|         54/476-1|      Feng Qian 0001|Proceedings of th...|2024|
|         204/4378|      Daniel Silhavy|3GPP Rel-17 5G Me...|2023|
|         355/2053|        David Waring|3GPP Rel-17 5G Me...|2023|
|         02/11004|          Dev Audsin|3GPP Rel-17 5G Me...|2023|
|         355/3143|    Richard Bradbury|3GPP Rel-17 5G Me...|2023|
|         325/4666|         Johann Mika|3GPP Rel-17 5G Me...|2023|
|         325/4623|   Klaus Kuehnhammer|3GPP Rel-17 5G Me...|2023|
|         196/5823|         Kurt Krauss|3GPP Rel-17 5G Me...|2023|
|         80/10608|  Jordi Joan Gimenez|3GPP Rel-17 5G Me...|2

In [44]:
spark.sql("SELECT a.pid, a.author, p.title, p.year FROM authors a INNER JOIN publications p ON array_contains(p.authors_pid, pid)").show()

+--------+--------------------+--------------------+----+
|     pid|              author|               title|year|
+--------+--------------------+--------------------+----+
| 33/7938|        Azeem Irshad|SEMS-5G: A Secure...|2024|
|131/6795| Mohammed Alreshoodi|SEMS-5G: A Secure...|2024|
|360/5884|Humphrey Owuor Ot...|Deployment and Ma...|2024|
| 00/8282|       Bessie Malila|Deployment and Ma...|2024|
| 22/8334|   Joyce B. Mwangama|Deployment and Ma...|2024|
|192/5382|     Sadiq ur Rehman|Beamforming Techn...|2024|
|214/9034|        Jawwad Ahmad|Beamforming Techn...|2024|
| 34/7195|       Anwaar Manzar|Beamforming Techn...|2024|
| 45/2434|  Muhammad Moinuddin|Beamforming Techn...|2024|
|296/1071|         Jim Bartels|A neural network-...|2024|
|162/6872|  Korkut Kaan Tokgoz|A neural network-...|2024|
| 34/5004|         Shiro Dosho|A neural network-...|2024|
|85/10282|    Toshihide Suzuki|A neural network-...|2024|
| 09/4332|        Hiroyuki Ito|A neural network-...|2024|
|156/4890|Arav

In [52]:
a = spark.sql("SELECT a.pid, a.author, p.title, p.year FROM authors a INNER JOIN publications p ON array_contains(p.authors_pid, pid)")
all_data = [(i.pid, i.author, i.title, i.year) for i in a.collect()]


[('33/7938', 'Azeem Irshad', 'SEMS-5G: A Secure and Efficient Multi-Server Authentication Scheme for 5G Networks.', '2024'), ('131/6795', 'Mohammed Alreshoodi', 'SEMS-5G: A Secure and Efficient Multi-Server Authentication Scheme for 5G Networks.', '2024'), ('360/5884', 'Humphrey Owuor Otieno', 'Deployment and Management of Intelligent End-to-End Network Slicing in 5G and Beyond 5G Networks: A Systematic Review.', '2024'), ('00/8282', 'Bessie Malila', 'Deployment and Management of Intelligent End-to-End Network Slicing in 5G and Beyond 5G Networks: A Systematic Review.', '2024'), ('22/8334', 'Joyce B. Mwangama', 'Deployment and Management of Intelligent End-to-End Network Slicing in 5G and Beyond 5G Networks: A Systematic Review.', '2024'), ('192/5382', 'Sadiq ur Rehman', 'Beamforming Techniques for MIMO-NOMA for 5G and Beyond 5G: Research Gaps and Future Directions.', '2024'), ('214/9034', 'Jawwad Ahmad', 'Beamforming Techniques for MIMO-NOMA for 5G and Beyond 5G: Research Gaps and Fut

In [60]:
from collections import defaultdict

title_to_authors = defaultdict(list)
for _, author, title, _ in all_data:
    title_to_authors[title].append(author)

defaultdict(<class 'list'>, {'SEMS-5G: A Secure and Efficient Multi-Server Authentication Scheme for 5G Networks.': ['Azeem Irshad', 'Mohammed Alreshoodi'], 'Deployment and Management of Intelligent End-to-End Network Slicing in 5G and Beyond 5G Networks: A Systematic Review.': ['Humphrey Owuor Otieno', 'Bessie Malila', 'Joyce B. Mwangama'], 'Beamforming Techniques for MIMO-NOMA for 5G and Beyond 5G: Research Gaps and Future Directions.': ['Sadiq ur Rehman', 'Jawwad Ahmad', 'Anwaar Manzar', 'Muhammad Moinuddin'], 'A neural network-based DPD coefficient determination for PA linearization in 5G and beyond-5G mmWave systems.': ['Jim Bartels', 'Korkut Kaan Tokgoz', 'Shiro Dosho', 'Toshihide Suzuki', 'Hiroyuki Ito', 'Aravind Tharayil Narayanan', 'Ludovico Minati', 'Aran Hagihara', 'Jun Kobayashi', 'Toshihiro Shimura', 'Yoichi Kawano', 'Parthojit Chakraborty'], '5G enabled smart cities: A real-world evaluation and analysis of 5G using a pilot smart city application.': ['Abhik Banerjee', 'Bre

In [61]:
import networkx as nx
import plotly.graph_objects as go
from itertools import combinations

def create_graph_for_title(title):
    authors = title_to_authors[title]
    G = nx.Graph()
    if len(authors) > 1:
        for author1, author2 in combinations(authors, 2):
            G.add_edge(author1, author2)

    # Zastosowanie layoutu "spring"
    pos = nx.spring_layout(G, k=0.5, iterations=50)

    # Przygotowanie węzłów do rysowania
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color='gray'),
        hoverinfo='none',
        mode='lines'
    )

    node_x = []
    node_y = []
    node_text = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=node_text,
        textposition="top center",
        hoverinfo='text',
        marker=dict(
            size=20,
            color='skyblue',
            line_width=2
        )
    )

    # Tworzenie figury
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=title,
                        title_x=0.5,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False),
                        autosize=True
                    ))

    return fig

In [62]:
import dash
from dash import html, dcc

app = dash.Dash(__name__)

# Layout aplikacji
app.layout = html.Div([
    html.H1("Interaktywny graf współautorstwa książek"),
    
    # Dropdown dla tytułów
    dcc.Dropdown(
        id='title-dropdown',
        options=[{'label': title, 'value': title} for title in title_to_authors.keys()],
        value=list(title_to_authors.keys())[0],  # Domyślny tytuł
        style={'width': '50%', 'padding': '10px', 'margin': '10px auto'}
    ),
    
    # Miejsce na wykres
    dcc.Graph(id='graph', style={'height': '80vh', 'width': '100%'})
])

In [63]:
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('title-dropdown', 'value')]
)
def update_graph(selected_title):
    return create_graph_for_title(selected_title)

# Uruchomienie aplikacji
app.run_server(debug=True)